In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
dev=qml.device('default.mixed',wires=3)
proj = np.array([[1, 0], [0, 0]])
@qml.qnode(dev)
def circuit1(params,p):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    
    qml.U3(params[0],params[1],params[2],wires=1)
    qml.U3(params[3],params[4],params[5],wires=2)
    qml.CNOT(wires=[2,1])
    qml.RZ(params[6],wires=1)
    qml.RY(params[7],wires=2)
    qml.CNOT(wires=[1,2])
    qml.RY(params[8],wires=2)
    qml.CNOT(wires=[2,1])
    qml.U3(params[9],params[10],params[11],wires=1)
    qml.U3(params[12],params[13],params[14],wires=2)
    
    qml.DepolarizingChannel(1-p,wires=1)
    qml.DepolarizingChannel(1-p,wires=2)
    
    qml.adjoint(qml.U3(params[12],params[13],params[14],wires=2))
    qml.adjoint(qml.U3(params[9],params[10],params[11],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.RY(params[8],wires=2))
    qml.CNOT(wires=[1,2])
    qml.adjoint(qml.RY(params[7],wires=2))
    qml.adjoint(qml.RZ(params[6],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.U3(params[3],params[4],params[5],wires=2))
    qml.adjoint(qml.U3(params[0],params[1],params[2],wires=1))

    qml.QubitUnitary(proj, wires=2)

    return qml.density_matrix(wires=[0,1])

def fidelity(state, target):
    target_dm = np.outer(target, np.conj(target))
    fidd = np.abs(np.dot(target.conj().T, np.dot(state, target)))
    fid=fidd/np.trace(state)
    return np.real(fid)
target_state = np.array([1, 0, 0, 1]) / np.sqrt(2)
def cost(params,p):
    return 1-fidelity(circuit1(params,p), target_state)
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)

In [ ]:
p_values = [0.25,0.3,0.35,0.40,0.45,0.50,0.55, 0.60,0.65,0.70,0.75,0.80, 0.85,0.90,0.95, 1]
num_seeds = 5

# Storage for fidelities and success probabilities for each p value and seed
all_fidelities = []
all_success_probabilities = []

# Optimization loop
for p in p_values:

    fid_init=0
    p_sucss_init=0
    for seed in range(num_seeds):
        np.random.seed(seed)
        params = np.random.uniform(-np.pi,np.pi,15)
        for i in range(150):
            params = optimizer.step(lambda v: cost(v, p), params)
        circuit_output = circuit1(params, p)
        current_fidelity = fidelity(circuit_output, target_state)
        current_success_prob = np.trace(circuit_output).real  # Assuming circuit1 returns a density matrix
        if current_fidelity> fid_init:
            fid_init=current_fidelity
            p_sucss_init= current_success_prob
    final_fid=fid_init
    final_p=p_sucss_init
    all_fidelities.append(final_fid)
    all_success_probabilities.append(final_p)
print(all_fidelities)
print(all_success_probabilities)